#Import

In [ ]:
!pip install transformers
!pip install tokenizers
!pip install Datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 26.1 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 6.6 MB 56.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 27.7 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 115 kB 73.4 MB/s 
     |████████████████████████████████| 127 kB 74.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer

import transformers
from transformers import (
    BertModel,
    BertTokenizer,
    AdamW,
    get_linear_schedule_with_warmup,
    BertConfig,
    BertForMaskedLM,
    Trainer,
    TrainingArguments)

import torch

import numpy as np
import pandas as pd
import os
import random

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Set up all seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [ ]:
df_train = pd.read_csv("/content/gdrive/MyDrive/data/Bofore_Peak_for_Pre-train.csv",index_col=0 )

In [ ]:
# # for data train data
# def split_sequence(series):
#     seq_split = []
#     for i in series:
#         temp = ""
#         temp2 = 0
#         for j in i :
#             if temp2 == 3 :
#                 temp2 = 0
#                 temp = temp + " "
#                 if len(temp.split()) == 512:
#                     seq_split.append(temp)
#                     temp = ""
#             else :
#                 temp = temp + j
#                 temp2 += 1

#     return seq_split

# train_data = split_sequence(df_train['Sequence'][0:70])
# vali_data = split_sequence(df_train['Sequence'][70:])

In [ ]:
def getKmers(sequence, size=3):
  seq = ""
  for x in range(len(sequence) - size + 1):
    seq += sequence[x:x+size] + " "
  return seq

In [ ]:
size = 3
cal = size - 1 # formula is n+5
def split_seq(series):
    seq_split = []
    for i in series:
        for j in range(1,(len(i)//(512+cal))+1):
            seq_split.append(getKmers(i[(512+cal)*(j-1):(512+cal)*j]))

    return seq_split

In [ ]:
# def split_seq(series):
#     seq_split = []
#     for i in series:
#         lit_seq = getKmers(i)
#         for j in range(1,(len(i)//512) +1 ):
#             seq_split.append(lit_seq[:512*j])

#     return seq_split

In [ ]:
train_data = split_seq(df_train['Sequence'][0:70])
vali_data = split_seq(df_train['Sequence'][70:])

In [ ]:
len(train_data), len(vali_data)

(4185, 1786)

In [ ]:
len(train_data), len(vali_data)

(980, 420)

# Build model

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_3")

model = AutoModelForMaskedLM.from_pretrained("armheb/DNA_bert_3")
count_parameter = model.num_parameters()
print(f"BERT model has {model.num_parameters()/10**6}M parameters")

BERT model has 86.095941M parameters


## Import pretrain

In [ ]:
# text = ("GCTCGT ACGTGG CTTTGG AGACTC CGTGGA [MASK] ATCAGA GGCACG TCAACA TCTTAA AGATGG CACTTG")
inputs = tokenizer(train_data, return_tensors='pt',truncation=True,
                    padding='max_length',max_length=512) # what is return_tensors argumennt # max_lenght = 4976

valis = tokenizer(vali_data, return_tensors='pt',truncation=True,
                    padding='max_length',max_length=512)

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
valis['labels'] = valis.input_ids.detach().clone()

In [ ]:
inputs

{'input_ids': tensor([[ 2, 62, 43,  ..., 23, 15,  3],
        [ 2, 42, 28,  ...,  5,  6,  3],
        [ 2, 10, 28,  ..., 17, 56,  3],
        ...,
        [ 2, 10, 25,  ..., 54, 10,  3],
        [ 2, 14, 44,  ..., 58, 27,  3],
        [ 2, 53,  5,  ..., 15, 45,  3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ 2, 62, 43,  ..., 23, 15,  3],
        [ 2, 42, 28,  ...,  5,  6,  3],
        [ 2, 10, 28,  ..., 17, 56,  3],
        ...,
        [ 2, 10, 25,  ..., 54, 10,  3],
        [ 2, 14, 44,  ..., 58, 27,  3],
        [ 2, 53,  5,  ..., 15, 45,

## Masking  data

In [ ]:
def masking_data(tokenize_data):
    # set seed torch
    torch.manual_seed(RANDOM_SEED)

    rand = torch.rand(tokenize_data.input_ids.shape)

    # where the random array is less than 0.15, we set true # except 2 ,3 because it is [SEP] & [CLS]
    mask_arr = (rand < 0.15) * (tokenize_data.input_ids != 2) * (tokenize_data.input_ids != 3) * (tokenize_data.input_ids != 0) # set 0.15 is probability to masking model

    # find selection number in list type for use to blind inputs data
    selection = []
    for i in range(mask_arr.shape[0]): # (100)
        selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

    # blind data from selection
    for i in range(mask_arr.shape[0]): # (100)
        tokenize_data.input_ids[i, selection[i]] = 4

In [ ]:
masking_data(inputs)
masking_data(valis)

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)
dataset_2 = MeditationsDataset(valis)

##Train

In [ ]:
batch_size = 16
model_name = "dna_bert_3_2"
logging_steps = (len(inputs.input_ids)) // batch_size

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # push_to_hub=True, # push to huggingface
    fp16=True,# for GPU
	logging_steps=logging_steps,
    num_train_epochs=10,
    seed=42
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset_2
    # compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


In [ ]:
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4185
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2620
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Epoch,Training Loss,Validation Loss
1,0.124500,0.120988
2,0.027300,0.096615
3,0.022100,0.093954
4,0.020200,0.092160
5,0.019500,0.088780
6,0.019000,0.089210
7,0.018500,0.089209
8,0.018600,0.086757
9,0.018100,0.087669
10,0.018200,0.087161


***** Running Evaluation *****
  Num examples = 1786
  Batch size = 16
Saving model checkpoint to dna_bert_3_2-finetuned/checkpoint-500
Configuration saved in dna_bert_3_2-finetuned/checkpoint-500/config.json
Model weights saved in dna_bert_3_2-finetuned/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
***** Running Evaluation *****
  Num examples = 1786
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1786
  Batch size = 16
Saving model checkpoint to dna_bert_3_2-finetuned/checkpoint-1000
Configuration saved in dna_bert_3_2-finetuned/checkpoint-1000/config.json
Model weights saved in dna_bert_3_2-finetuned/checkpoint-1000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy

CPU times: user 23min 18s, sys: 5min 25s, total: 28min 44s
Wall time: 29min 2s


TrainOutput(global_step=2620, training_loss=0.030542878863465694, metrics={'train_runtime': 1742.459, 'train_samples_per_second': 24.018, 'train_steps_per_second': 1.504, 'total_flos': 1.10112062805504e+16, 'train_loss': 0.030542878863465694, 'epoch': 10.0})

##Evaluation & analyze result

### Perplexity

In [ ]:
import math
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1786
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


>>> Perplexity: 1.09


### Bleu score & f1-score

In [ ]:
!pip install nltk
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.7 MB/s 


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
predictions = trainer.predict(dataset_2)
preds = np.argmax(predictions.predictions, axis=-1)

***** Running Prediction *****
  Num examples = 1786
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
# find average bleu score & f1-score
import evaluate
f1_metric = evaluate.load("f1")
lit_bleu = []
lit_f1 = []

for i in range(len(preds)):
  candidate = preds[i].tolist() # 512
  references = [predictions.label_ids[i].tolist()]

  lit_f1.append(f1_metric.compute(predictions=preds[i].tolist(),
                                  references=predictions.label_ids[i].tolist(),
                                  average="weighted")['f1'])

  lit_bleu.append(sentence_bleu(references, candidate))

print(f"bleu score: {(sum(lit_bleu)/len(lit_bleu)):.4f}")
print(f"f1-score: {(sum(lit_f1)/len(lit_f1)):.4f}")

bleu score: 0.9829
f1-score: 0.9862


### collect model parameters

In [ ]:
model.num_parameters()

86095941

#PushToHub

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to dna_bert_3_2-finetuned
Configuration saved in dna_bert_3_2-finetuned/config.json
Model weights saved in dna_bert_3_2-finetuned/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/328M [00:00<?, ?B/s]

Upload file runs/Aug09_18-50-50_98ac3fdafb4e/events.out.tfevents.1660071070.98ac3fdafb4e.74.0:   7%|7         …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Mozart-coder/dna_bert_3_2-finetuned
   e685902..58b54d0  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/Mozart-coder/dna_bert_3_2-finetuned
   58b54d0..163a473  main -> main



'https://huggingface.co/Mozart-coder/dna_bert_3_2-finetuned/commit/58b54d0ce0d0da6e59c9fbb9d6da2bbcc588c854'